In [1]:
import pandas as pd
import plotly.express as px

# Getaround EDA : analyse exploratoire des données

Cette analyse de données doit permettre de fournir les arguments pour répondre aux deux questions suivantes en proposant une réponse à chacune.
- **seuil** : quelle doit être la durée minimale du délai entre deux locations ?
- **périmètre** : faut-il activer cette fonctionnalité pour tous les véhicules ou uniquement pour les véhicules Connect ?

On peut également approfondir l'exploration en s'appuyant sur les questions :
- **impact** : quel est l'impact de cette fonctionnalité sur le nombre de locations ?
- Quelle part des revenus des propriétaires serait potentiellement affectée par cette fonctionnalité ?
- Combien de locations seraient impactées en fonction du seuil et du périmètre choisis ?
- À quelle fréquence les conducteurs sont-ils en retard pour le check-in suivant ? Quel est l’impact pour le conducteur suivant ?
- Combien de situations problématiques seraient résolues selon le seuil et le périmètre retenus ?



## Analyse

| field name	|   Comment	|   
| ----------- | ----------- |
| rental_id	|       Unique identifier of the rental	|   
| car_id	    |       Unique identifier of the car	|   
| checkin_type |   	Flow used for both checkin and checkout. (ie. access and return the car) <br> mobile = rental agreement signed on the owner's smartphone <br> connect = car equiped with the Connect technology , opened by the driver with his smartphone.<br> **Note:** paper contracts were excluded from the data as we have no data on their delay at checkout and it's negligible use case |   
| state	   |        canceled means that the rental did not happen (was canceled by the driver or the owner).	|   
| delay_at_checkout_in_minutes	|    Difference in minutes between the rental end time requested by the driver when booking the car and the actual time the driver completed the checkout. Negative values |mean that the driver returned the car in advance.|   
| previous_ended_rental_id	|    id of the previous ended rental of the car (NULL when no previous rental or delay with previous rental higher than 12 hours)	|   
| time_delta_with_previous_rental_in_minutes	|    Difference in minutes between this rental planned start time and the previous rental planned end time (when lower than 12 hours, NULL if higher)	|   
        

In [2]:
df=pd.read_csv("../data/raw/get_around_delay_analysis.csv",sep=";")

In [3]:
display(df.describe())
display(df.info())
display(df.head(10))

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Unnamed: 7,Unnamed: 8
count,21310.000000,21310.000000,16346.000000,1841.000000,1841.000000,0.0,0.0
mean,549712.880338,350030.603426,59.701517,550127.411733,279.288430,NaN,NaN
std,13863.446964,58206.249765,1002.561635,13184.023111,254.594486,NaN,NaN
min,504806.000000,159250.000000,-22433.000000,505628.000000,0.000000,NaN,NaN
25%,540613.250000,317639.000000,-36.000000,540896.000000,60.000000,NaN,NaN
50%,550350.000000,368717.000000,9.000000,550567.000000,180.000000,NaN,NaN
75%,560468.500000,394928.000000,67.000000,560823.000000,540.000000,NaN,NaN
max,576401.000000,417675.000000,71084.000000,575053.000000,720.000000,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
 7   Unnamed: 7                                  0 non-null      float64
 8   Unnamed: 8                                  0 non-null      float64
dtypes: float64(5), int64(2), object(2)
memory usage: 1.5+ MB


None

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Unnamed: 7,Unnamed: 8
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN,NaN,NaN
5,511626,398802,mobile,ended,-203.0,NaN,NaN,NaN,NaN
6,511639,370585,connect,ended,-15.0,563782.0,570.0,NaN,NaN
7,512303,371242,mobile,ended,-44.0,NaN,NaN,NaN,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN,NaN,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN,NaN,NaN


In [4]:
fig = px.histogram(df, x="delay_at_checkout_in_minutes", nbins=100)
fig.update_traces(xbins=dict( # bins used for histogram
        start=-200.0,
        end=200.0,
        size=5
    ))
fig.show()

In [5]:
df_ended = df[df["state"]=="ended"][["rental_id","delay_at_checkout_in_minutes"]]
fig = px.histogram(df_ended, x="delay_at_checkout_in_minutes", nbins=100)
fig.update_traces(xbins=dict( # bins used for histogram
        start=-200.0,
        end=200.0,
        size=5
    ))
fig.show()  

In [6]:
df[df["state"]=="canceled"]

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Unnamed: 7,Unnamed: 8
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN,NaN,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN,NaN,NaN
10,513743,330658,mobile,canceled,NaN,NaN,NaN,NaN,NaN
11,514161,366037,connect,canceled,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21283,569325,345079,mobile,canceled,NaN,NaN,NaN,NaN,NaN
21287,569764,405347,mobile,canceled,NaN,NaN,NaN,NaN,NaN
21288,570001,386413,connect,canceled,NaN,NaN,NaN,NaN,NaN
21297,571481,311841,mobile,canceled,NaN,NaN,NaN,NaN,NaN


In [7]:
df_canceled = df[df["state"]=="canceled"][["rental_id","delay_at_checkout_in_minutes"]]
fig = px.histogram(df_canceled, x="delay_at_checkout_in_minutes", nbins=100)

fig.show()  
df_cancelled

NameError: name 'df_cancelled' is not defined

In [ ]:
df_canceled = df[(df["delay_at_checkout_in_minutes"] >= -60) & (df["delay_at_checkout_in_minutes"] <= 60)]  

In [ ]:
df_canceled 

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Unnamed: 7,Unnamed: 8
6,511639,370585,connect,ended,-15.0,563782.0,570.0,NaN,NaN
7,512303,371242,mobile,ended,-44.0,NaN,NaN,NaN,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN,NaN,NaN
13,515147,257466,mobile,ended,15.0,NaN,NaN,NaN,NaN
19,519491,312389,mobile,ended,58.0,545639.0,420.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21292,570440,292303,mobile,ended,32.0,NaN,NaN,NaN,NaN
21293,570957,390479,mobile,ended,7.0,NaN,NaN,NaN,NaN
21294,571154,400293,mobile,ended,2.0,NaN,NaN,NaN,NaN
21296,571359,357612,mobile,ended,52.0,NaN,NaN,NaN,NaN


In [ ]:
df_analysis=df[(df["state"]=="canceled") & df["time_delta_with_previous_rental_in_minutes"]>0 ]

In [ ]:
df_analysis[["rental_id","car_id","checkin_type","state","time_delta_with_previous_rental_in_minutes"]]

,rental_id,car_id,checkin_type,state,time_delta_with_previous_rental_in_minutes
204,543768,374169,connect,canceled,210.0
242,546160,352528,connect,canceled,630.0
504,564627,341431,mobile,canceled,150.0
637,568657,317378,connect,canceled,210.0
669,516550,377700,mobile,canceled,720.0
...,...,...,...,...,...
21022,560787,413181,mobile,canceled,150.0
21172,566228,390871,connect,canceled,60.0
21230,569706,245154,connect,canceled,660.0
21269,568049,381499,connect,canceled,720.0


In [ ]:
fig = px.histogram(df_analysis, x="time_delta_with_previous_rental_in_minutes", nbins=200)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0,
        end=1000.0,
        size=30
    ))
fig.show()